In [1]:
from data_sampler import WeightedSampler
from dataset import CARLADataset, CARLADatasetMultiProcessing
from data_preprocessing import preprocessing
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
import torch
import os
import pandas as pd
import matplotlib.pyplot as plt

/Users/julianvonklitzing/miniforge3/envs/carla/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
not [1]

False

In [48]:
df = pd.DataFrame()

In [52]:
df.empty

False

In [43]:
df.columns.__contains__("bla")

True

In [51]:
df["bla"] = [1, 2, 3]

In [37]:
df

,bla
0,1
1,2
2,3


In [45]:
"../data/Dataset Ege/Town10HD_Scenario10_route16_11_28_18_26_19".split(os.sep)

['..', 'data', 'Dataset Ege', 'Town10HD_Scenario10_route16_11_28_18_26_19']

In [58]:
df = pd.DataFrame()
df_temp = pd.DataFrame()
for (root,dirs,files) in os.walk('../data/Dataset Ege', topdown=True):
    # print("Root: ", root)
    # print("Dirs: ", dirs)
    # print("Files: ", files)
    # print('--------------------------------')
    
    if not dirs:
        input_type = root.split(os.sep)[-1]
        if df_temp.columns.__contains__(input_type):
            if df.empty:
                df = df_temp
            else:
                df = pd.concat([df, df_temp], axis=1, ignore_index=True)
            df_temp = pd.DataFrame()
        else:
            if df_temp.empty:
                df_temp["dir"] = [root] * len(files)
            df_temp[input_type] = sorted(files)

In [59]:
df

,dir,topdown,label_raw,depth,rgb,semantics,lidar,measurements
0,../data/Dataset Ege/Town10HD_Scenario10_route1...,encoded_0000.png,0000.json,0000.png,0000.png,0000.png,0000.npy,0000.json
1,../data/Dataset Ege/Town10HD_Scenario10_route1...,encoded_0001.png,0001.json,0001.png,0001.png,0001.png,0001.npy,0001.json
2,../data/Dataset Ege/Town10HD_Scenario10_route1...,encoded_0002.png,0002.json,0002.png,0002.png,0002.png,0002.npy,0002.json
3,../data/Dataset Ege/Town10HD_Scenario10_route1...,encoded_0003.png,0003.json,0003.png,0003.png,0003.png,0003.npy,0003.json
4,../data/Dataset Ege/Town10HD_Scenario10_route1...,encoded_0004.png,0004.json,0004.png,0004.png,0004.png,0004.npy,0004.json
...,...,...,...,...,...,...,...,...
59,../data/Dataset Ege/Town10HD_Scenario10_route1...,encoded_0059.png,0059.json,0059.png,0059.png,0059.png,0059.npy,0059.json
60,../data/Dataset Ege/Town10HD_Scenario10_route1...,encoded_0060.png,0060.json,0060.png,0060.png,0060.png,0060.npy,0060.json
61,../data/Dataset Ege/Town10HD_Scenario10_route1...,encoded_0061.png,0061.json,0061.png,0061.png,0061.png,0061.npy,0061.json
62,../data/Dataset Ege/Town10HD_Scenario10_route1...,encoded_0062.png,0062.json,0062.png,0062.png,0062.png,0062.npy,0062.json


In [25]:
os.listdir("../data/data/cycl_dataset_23_11/Routes_non-straight-junction-Scen4_Town07_junction_Seed2000/non-straight-junction-Scen4_Town07_junction_route28_11_23_22_00_23")

['topdown', 'label_raw', 'depth', 'rgb', 'semantics', 'lidar', 'measurements']

In [22]:
"measurements" in os.listdir("../data/data/cycl_dataset_23_11")

False

In [13]:
path_ege_data = os.path.join("..", "data", "Dataset Ege")

config = {"used_inputs": ["rgb", "lidar", "measurements"], 
        "used_measurements": ["speed", "steer", "throttle"],
        "seq_len": 1
        }

dataset = CARLADataset(root_dir=path_ege_data, config=config)
dataset_mp = CARLADatasetMultiProcessing(root_dir=path_ege_data, config=config)
weighted_sampler = WeightedSampler(dataset=dataset)
print(dataset.__len__())

217


In [14]:
dl = DataLoader(dataset=dataset, batch_size=16, num_workers=0, sampler=weighted_sampler)
dl_mp = DataLoader(dataset=dataset_mp, batch_size=16, num_workers=3, sampler=weighted_sampler)

In [4]:
count = 0
for batch in dl:
    # preprocessing
    for key in preprocessing:
        batch[key] = preprocessing[key](batch[key])
    count += 1
print(count)

14


In [5]:
# Derive statistics for our dataset that can be used in Normalize() method to get approx. zero mean & var 1. 
mean = 0
std = 0
count = 0
for batch in dl:
    mean = (count * mean + batch["rgb"].shape[0] * batch["rgb"].mean(axis=[0, 1, 3, 4])) / (count + batch["rgb"].shape[0])
    std = (count * std + batch["rgb"].shape[0] * batch["rgb"].std(axis=[0, 1, 3, 4])) / (count + batch["rgb"].shape[0])
    count += batch["rgb"].shape[0]
print(count)

217


In [15]:
dataset.__getitem__(1)

ValueError: could not broadcast input array from shape (27352,4) into shape (27498,4)